In [1]:
# Create GUI elements
import tkinter
import time
import os
from tkinter import messagebox
import random
import mysql.connector as mariadb

# Open database connection
db = mariadb.connect(user="root",password="root",database="reminders" )

# prepare a cursor object using cursor() method
cursor = db.cursor()

# execute SQL query using execute() method.
cursor.execute("SELECT VERSION()")

# Fetch a single row using fetchone() method.
data = cursor.fetchone()
print ("Database version : %s " % data)

#Create root window
root = tkinter.Tk()

# Change root window bg colour
root.configure(bg="white")

# Create title
root.title("Reminders")

# set window size
root.geometry("500x500")

#Create empty list
tasks = []
locations= []
times= []


sql = "select * from reminders_table"
try:
    cursor.execute(sql)
    results = cursor.fetchall()
    for row in results:
        message = row[0]
        t= row[1]
        location = row[2]
        tasks.append(message)
        locations.append(location)
        times.append(t)
except:
    print ("Error: unable to fetch data")
    

def checkTodaysReminders(): 
    today = time.strftime('%d/%m/%y')
    print(today)
    sql = "select * from reminders_table"
    cursor.execute(sql)
    results = cursor.fetchall()
    flag = 0
    for row in results:
        message = row[0]
        t = row[1]
        location = row[2]
        if t in times:
            if t == today:                 
                flag =1
                messagebox.showinfo("Reminder of the day!",'"Reminder " ' + message + ' ' + location + '"')
                
    if flag == 0: 
            messagebox.showinfo( "Reminder of the day!","No Reminders!") 
    
    
# Create functions:

checkTodaysReminders()
def update_listbox():
    # clear the current listbox
    clear_listbox()
    clear_listbox1()
    clear_listbox2()
    # Populate listbox by appending each task to list
    for task in tasks:
        lb_tasks.insert("end", task)
    for loca in locations:
        lb_location.insert("end",loca)
    for tm in times:
        lb_times.insert("end",tm)
        
def clear_listbox():
    lb_tasks.delete(0, "end")
def clear_listbox1():
    lb_location.delete(0,"end")
def clear_listbox2():
    lb_times.delete(0,"end")

def add_task(event=None): # "event=None" so that enter key can add task without clicking the button
    #get user input(prompt)
    task = txt_input.get()
    print(task)
    loca=loc_input.get()
    tm=tym_input.get()
    # Ensure user has enetered a task
    if task !="":
        tasks.append(task)
        locations.append(loca)
        times.append(tm)
        update_listbox()
        sql="insert into reminders_table(message,time,location) values ('"+ task +"','"+ tm +"','"+ loca +"');"
        print('sql insert', sql)
        cursor.execute(sql)
        db.commit()
        update_listbox()

    else:
        messagebox.showwarning("Note!", "Please enter a task")
    # Clear the textbox to avoid adding the same task twice accidentally
    txt_input.delete(0, "end")
    loc_input.delete(0, "end")
    tym_input.delete(0, "end")

root.bind('<Return>', add_task)# bind return key to add_task so that enter key can add task without clicking the button



def delete_task():

    # Get the text of the currently selected item
    task = lb_tasks.get("active")
    loca = lb_location.get("active")
    tm=lb_times.get("active")
    # Confirm task is in list
    if task in tasks:
        confirm_del = messagebox.askyesno("Confirm Deletion", "Are you sure you want to delete task:   ** {} ** ?".format(task))
        if confirm_del:
            sql = 'delete from reminders_table where message="'+ task + '";'
            print('sql delete', sql)
            cursor.execute(sql)
            db.commit()
            tasks.remove(task)
            update_listbox()
    if loca in locations:
        confirm_del = messagebox.askyesno("Confirm Deletion", "Are you sure you want to delete location:   ** {} ** ?".format(loca))
        if confirm_del:
            sql = 'delete from reminders_table where location="'+ loca + '";'
            print('sql delete', sql)
            cursor.execute(sql)
            db.commit()
            locations.remove(loca)
            update_listbox()
    if tm in times:
        confirm_del = messagebox.askyesno("Confirm Deletion", "Are you sure you want to delete time:   ** {} ** ?".format(tm))
        if confirm_del:
            sql = 'delete from reminders_table where location="'+ tm + '";'
            print('sql delete', sql)
            cursor.execute(sql)
            db.commit()
            times.remove(tm)
            update_listbox()


def delete_all():
    # As list is being changed, it needs to be global.
    global tasks
    global locations
    global times
    confirm_del = messagebox.askyesno("Delete All Confirmation", "Are you sure you want to delete all tasks & locations?")
    if confirm_del:
        sql = 'delete from reminders_table;'
        print('sql delete', sql)
        cursor.execute(sql)
        db.commit()
      # Clear the tasks and locations list.
        tasks = []
        locations =[]
        times=[]
      # Update listbox
        update_listbox()
        
root.bind('<Return>', add_task)

# Create title in root widget of GUI with white background
lbl_title = tkinter.Label(root, text="Reminder", bg="white")
lbl_title.grid(row= 0, column=0 )

lbl_display = tkinter.Label(root, text="", fg="white")
lbl_display.grid(row=0 , column=1 )

lbl_title = tkinter.Label(root, text="Message", fg="black")
lbl_title.grid(row= 0, column=1 )
txt_input = tkinter.Entry(root, width = 15)
txt_input.grid(row=1 , column=1 )


lbl_title2 = tkinter.Label(root, text="Time in DD-MM-YYYY", fg="black")
lbl_title2.grid(row= 0, column=2 )
tym_input=tkinter.Entry(root, width = 15)
tym_input.grid(row=1 , column=2)


lbl_title3 = tkinter.Label(root, text="Location", fg="black")
lbl_title3.grid(row= 0, column=3 )
loc_input=tkinter.Entry(root, width = 15)
loc_input.grid(row=1 , column=3)



btn_add_task = tkinter.Button(root, text="Add Task", fg="green", bg="white", command=add_task)
btn_add_task.grid(row= 1, column=0 )


btn_delete_task = tkinter.Button(root, text="Delete Task", fg="green", bg="white", command=delete_task)
btn_delete_task.grid(row=2 , column=0 )

btn_delete_all = tkinter.Button(root, text="Delete All", fg="green", bg="white", command=delete_all)
btn_delete_all.grid(row=3 , column= 0)


lb_tasks = tkinter.Listbox(root)
lb_tasks.grid(row=2 , column=1, rowspan=5 )



lb_times=tkinter.Listbox(root)
lb_times.grid(row=2,column=2,rowspan=5)



lb_location=tkinter.Listbox(root)
lb_location.grid(row=2,column=3,rowspan=5)

#Populate listbox at program start for future file io functionality
def show_listbox():
    global tasks
    for task in tasks:
        lb_tasks.insert("end", task)
def show_listbox1():
    global locations
    for loca in locations:
        lb_location.insert("end",loca)
def show_listbox2():
    global times
    for tm in times:
        lb_times.insert("end",tm)
    
#Populate listbox at program start for future file io functionality
show_listbox()
show_listbox2()
show_listbox1()

    # Start the main events loop
 
    
root.mainloop()


Database version : 10.4.6-MariaDB 
25/07/19
